# <center>**Lab Sheet 10**</center>
# <center>**Real-World Experiments & Projects**</center>

In [1]:
# Saves plots to ./lab10_plots/
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc
import torch
import torch.nn as nn
import torch.optim as optim
np.random.seed(42)
torch.manual_seed(42)
os.makedirs("lab10_plots", exist_ok=True)

# ---------------------
# Helper utilities
# ---------------------
def to_torch(x, dtype=torch.float32):
    return torch.tensor(x, dtype=dtype)

**Q1. Digit classifier (0-9) using PyTorch (trained on sklearn digits)**

In [2]:

digits = load_digits()
X = digits.data         
y = digits.target

# small train/val/test split
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=1, stratify=y_trainval)

# standardize
scaler = StandardScaler().fit(X_train)
X_train_s = scaler.transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

# pytorch dataset
Xtr = torch.tensor(X_train_s, dtype=torch.float32)
ytr = torch.tensor(y_train, dtype=torch.long)
Xv  = torch.tensor(X_val_s, dtype=torch.float32)
yv  = torch.tensor(y_val, dtype=torch.long)
Xt  = torch.tensor(X_test_s, dtype=torch.float32)
yt  = torch.tensor(y_test, dtype=torch.long)

class SimpleMLP(nn.Module):
    def __init__(self, input_dim=64, hidden=64, n_classes=10, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, n_classes)
        )
    def forward(self,x): return self.net(x)

model = SimpleMLP(input_dim=64, hidden=64, n_classes=10, dropout=0.2)
opt = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

# train 
epochs = 80
train_losses, val_accs = [], []
for ep in range(epochs):
    model.train()
    opt.zero_grad()
    out = model(Xtr)
    loss = loss_fn(out, ytr)
    loss.backward()
    opt.step()
    train_losses.append(loss.item())

    model.eval()
    with torch.no_grad():
        val_pred = model(Xv).argmax(1).numpy()
    acc = accuracy_score(y_val, val_pred)
    val_accs.append(acc)
    if (ep+1) % 20 == 0:
        print(f"Epoch {ep+1}/{epochs} loss={loss.item():.4f} val_acc={acc:.4f}")

# evaluate on test
model.eval()
with torch.no_grad():
    test_logits = model(Xt)
    test_pred = test_logits.argmax(1).numpy()
test_acc = accuracy_score(y_test, test_pred)
test_f1  = f1_score(y_test, test_pred, average='macro')
print(f"Test accuracy={test_acc:.4f}, Macro-F1={test_f1:.4f}")

# save plots: loss + val acc
plt.figure()
plt.plot(train_losses)
plt.title("Training loss (digit MLP)")
plt.xlabel("epoch"); plt.ylabel("loss")
plt.savefig("lab10_plots/loss.png", dpi=150); plt.close()

plt.figure()
plt.plot(val_accs)
plt.title("Validation accuracy (digit MLP)")
plt.xlabel("epoch"); plt.ylabel("val acc")
plt.savefig("lab10_plots/valacc.png", dpi=150); plt.close()

# save some predicted digit images (first 12 test samples)
plt.figure(figsize=(8,6))
for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(X_test[i].reshape(8,8), cmap='gray')
    plt.title(f"pred:{test_pred[i]} gt:{y_test[i]}")
    plt.axis('off')
plt.suptitle("some test predictions")
plt.savefig("lab10_plots/sample_preds.png", dpi=150); plt.close()

Epoch 20/80 loss=0.1734 val_acc=0.9340
Epoch 40/80 loss=0.0553 val_acc=0.9653
Epoch 60/80 loss=0.0224 val_acc=0.9722
Epoch 80/80 loss=0.0162 val_acc=0.9792
Test accuracy=0.9722, Macro-F1=0.9719


**Q2. Neural net approximating sin(x) on [0,2π]**

In [3]:
N = 200
x = np.linspace(0, 2*np.pi, N).reshape(-1,1)
y_sin = np.sin(x)

# add some noise
y_noisy = y_sin + 0.08*np.random.randn(*y_sin.shape)

# train/val split
x_train, x_val, y_train_s, y_val_s = train_test_split(x, y_noisy, test_size=0.2, random_state=1)

# to tensors
Xt_s = torch.tensor(x_train, dtype=torch.float32)
Yt_s = torch.tensor(y_train_s, dtype=torch.float32)
Xv_s  = torch.tensor(x_val, dtype=torch.float32)
Yv_s  = torch.tensor(y_val_s, dtype=torch.float32)

class SineMLP(nn.Module):
    def __init__(self, hidden=50):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, 1)
        )
    def forward(self,x): return self.net(x)

model_sin = SineMLP(hidden=40)
opt_sin = optim.Adam(model_sin.parameters(), lr=0.01)
loss_fn_reg = nn.MSELoss()

epochs = 600
loss_hist = []
for ep in range(epochs):
    opt_sin.zero_grad()
    out = model_sin(Xt_s)
    loss = loss_fn_reg(out, Yt_s)
    loss.backward()
    opt_sin.step()
    loss_hist.append(loss.item())
    if (ep+1) % 150 == 0:
        print(f"Sine fit epoch {ep+1}, loss={loss.item():.6f}")

# evaluate on dense grid
x_grid = np.linspace(0, 2*np.pi, 300).reshape(-1,1)
with torch.no_grad():
    y_pred_grid = model_sin(torch.tensor(x_grid, dtype=torch.float32)).numpy()

plt.figure()
plt.plot(x_grid, np.sin(x_grid), label="true sin(x)")
plt.scatter(x_train, y_train_s, s=10, alpha=0.6, label="train (noisy)")
plt.plot(x_grid, y_pred_grid, label="MLP fit")
plt.legend(); plt.title("sin(x) approximation")
plt.savefig("lab10_plots/sin_fit.png", dpi=150); plt.close()

plt.figure()
plt.plot(loss_hist)
plt.title("Training loss")
plt.xlabel("epoch"); plt.ylabel("MSE")
plt.savefig("lab10_plots/loss.png", dpi=150); plt.close()

Sine fit epoch 150, loss=0.013560
Sine fit epoch 300, loss=0.005471
Sine fit epoch 450, loss=0.005219
Sine fit epoch 600, loss=0.005212


**Q3. Simple recurrent associative model (sequence Hopfield-style)**

In [4]:
def bipolar(v): return np.where(v>0, 1, -1)

# create some binary temporal patterns (short sequences of length L over D-dim)
D = 20   # dimension of pattern vectors
L = 6    # sequence length
num_sequences = 3
sequences = []
for s in range(num_sequences):
    seq = (np.random.rand(L, D) > 0.5).astype(int) * 1
    seq = bipolar(seq)   # convert to -1/+1
    sequences.append(seq)

# build transition matrix W such that W * x_t -> x_{t+1} (Hebbian heteroassociative)
W_trans = np.zeros((D, D))
for seq in sequences:
    for t in range(L-1):
        x_t = seq[t].reshape(-1,1)
        x_tp1 = seq[t+1].reshape(-1,1)
        W_trans += (x_tp1 @ x_t.T)   # outer product x_{t+1} * x_t^T
# Optional normalization
W_trans /= (num_sequences*(L-1))

# test recall: start from first vector and iterate
def recall_sequence(W, x0, steps=10, noisy_flip_frac=0.0):
    x = x0.copy()
    if noisy_flip_frac>0:
        idx = np.random.choice(len(x), size=int(len(x)*noisy_flip_frac), replace=False)
        x[idx] *= -1
    history = [x.copy()]
    for _ in range(steps):
        h = W @ x
        x = np.where(h>=0, 1, -1)
        history.append(x.copy())
    return history

# demonstrate for one stored sequence
seq0 = sequences[0]
start = seq0[0]
hist = recall_sequence(W_trans, start, steps=L-1, noisy_flip_frac=0.2)
# measure how many steps match target sequence elements
matches = [np.array_equal(hist[t], seq0[t]) for t in range(1, len(hist))]
print("match progression for seq0 (after noisy start):", matches)

# Save a small visualization: Hamming similarity over steps vs true sequence
similarities = [ (hist[t] == seq0[t]).mean() for t in range(len(hist)) ]  # fraction equal per position
plt.figure()
plt.plot(similarities, marker='o')
plt.title("Similarity to true sequence over recall steps")
plt.xlabel("step"); plt.ylabel("fraction matching")
plt.savefig("lab10_plots/sequence_similarity.png", dpi=150); plt.close()

match progression for seq0 (after noisy start): [False, False, False, False, False]


**Q4. Compare MLP vs SVM (accuracy, F1, ROC-AUC)**

In [5]:
# MLP predictions already in test_pred, test_logits
mlp_probs = torch.nn.functional.softmax(test_logits, dim=1).numpy()
mlp_pred = test_pred
mlp_acc = accuracy_score(y_test, mlp_pred)
mlp_f1  = f1_score(y_test, mlp_pred, average='macro')
# ROC-AUC (one-vs-rest)
y_test_bin = label_binarize(y_test, classes=np.arange(10))
mlp_roc = roc_auc_score(y_test_bin, mlp_probs, average='macro', multi_class='ovr')
print(f"MLP -> Acc: {mlp_acc:.4f}, Macro-F1: {mlp_f1:.4f}, Macro ROC-AUC: {mlp_roc:.4f}")

# SVM (with probability estimates)
svm = SVC(kernel='rbf', probability=True, gamma='scale', C=5.0, random_state=0)
svm.fit(X_train_s, y_train)
svm_pred = svm.predict(X_test_s)
svm_probs = svm.predict_proba(X_test_s)
svm_acc = accuracy_score(y_test, svm_pred)
svm_f1  = f1_score(y_test, svm_pred, average='macro')
svm_roc = roc_auc_score(y_test_bin, svm_probs, average='macro', multi_class='ovr')
print(f"SVM -> Acc: {svm_acc:.4f}, Macro-F1: {svm_f1:.4f}, Macro ROC-AUC: {svm_roc:.4f}")

# Plot ROC for class 0 vs rest as example
fpr_m, tpr_m, _ = roc_curve(y_test_bin[:,0], mlp_probs[:,0])
fpr_s, tpr_s, _ = roc_curve(y_test_bin[:,0], svm_probs[:,0])
plt.figure()
plt.plot(fpr_m, tpr_m, label=f"MLP class 0 AUC={auc(fpr_m,tpr_m):.3f}")
plt.plot(fpr_s, tpr_s, label=f"SVM class 0 AUC={auc(fpr_s,tpr_s):.3f}")
plt.plot([0,1],[0,1],'k--')
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (digit class 0 vs rest)")
plt.legend(); plt.savefig("lab10_plots/roc_class0.png", dpi=150); plt.close()

MLP -> Acc: 0.9722, Macro-F1: 0.9719, Macro ROC-AUC: 0.9983
SVM -> Acc: 0.9750, Macro-F1: 0.9749, Macro ROC-AUC: 0.9995


**Q5. Extract features (PCA) from images and feed to a shallow NN**

In [6]:
pca = PCA(n_components=20).fit(X_train_s)
X_train_pca = pca.transform(X_train_s)
X_test_pca  = pca.transform(X_test_s)

# small shallow NN (PyTorch) that takes PCA features
class ShallowNN(nn.Module):
    def __init__(self, input_dim, hidden=32, n_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, n_classes)
        )
    def forward(self,x): return self.net(x)

# train on PCA features
Xt_pca = torch.tensor(X_train_pca, dtype=torch.float32)
yt_pca = torch.tensor(y_train, dtype=torch.long)
Xt_pca_val = torch.tensor(pca.transform(X_val_s), dtype=torch.float32)

model_pca = ShallowNN(input_dim=20, hidden=48, n_classes=10)
opt_pca = optim.Adam(model_pca.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
for ep in range(120):
    opt_pca.zero_grad()
    out = model_pca(Xt_pca)
    loss = loss_fn(out, yt_pca)
    loss.backward(); opt_pca.step()
# eval on test
with torch.no_grad():
    test_pca_logits = model_pca(torch.tensor(X_test_pca,dtype=torch.float32))
    pred_pca = test_pca_logits.argmax(1).numpy()
acc_pca = accuracy_score(y_test, pred_pca)
print(f"PCA-based shallow NN test acc: {acc_pca:.4f}")

# also train shallow NN on raw pixels (for comparison)
Xt_raw = torch.tensor(X_train_s, dtype=torch.float32)
model_raw = ShallowNN(input_dim=64, hidden=48, n_classes=10)
opt_raw = optim.Adam(model_raw.parameters(), lr=0.01)
for ep in range(120):
    opt_raw.zero_grad()
    out = model_raw(Xt_raw)
    loss = loss_fn(out, yt_pca)  # note: yt_pca same labels
    loss.backward(); opt_raw.step()
with torch.no_grad():
    test_raw_logits = model_raw(torch.tensor(X_test_s,dtype=torch.float32))
    pred_raw = test_raw_logits.argmax(1).numpy()
acc_raw = accuracy_score(y_test, pred_raw)
print(f"raw-pixel shallow NN test acc: {acc_raw:.4f}")

# Plot comparison bar chart
plt.figure()
plt.bar(["PCA (20) NN", "Raw-pixel NN", "SVM", "MLP"], [acc_pca, acc_raw, svm_acc, mlp_acc])
plt.ylim(0,1); plt.ylabel("Accuracy"); plt.title("Feature extraction (PCA) vs raw vs baselines")
plt.savefig("lab10_plots/feature_comp.png", dpi=150); plt.close()

PCA-based shallow NN test acc: 0.9639
raw-pixel shallow NN test acc: 0.9722


# ---------------------
# Summary prints and saved plots
# ---------------------

In [7]:

print("MLP (digits) test acc, macro-F1:", test_acc, test_f1)
print("MLP vs SVM on digits ->")
print(f"  MLP -> Acc {mlp_acc:.4f}, F1 {mlp_f1:.4f}, ROC-AUC {mlp_roc:.4f}")
print(f"  SVM -> Acc {svm_acc:.4f}, F1 {svm_f1:.4f}, ROC-AUC {svm_roc:.4f}")
print("PCA-based shallow NN acc:", acc_pca, " raw-pixel NN acc:", acc_raw)

print("\nSaved plots (lab10_plots/):")
saved = [
 "loss.png", "valacc.png", "sample_preds.png",
 "sin_fit.png", "loss.png",
 "sequence_similarity.png",
 "roc_class0.png",
 "feature_comp.png"
]
for p in saved: print("lab10_plots/" + p)

MLP (digits) test acc, macro-F1: 0.9722222222222222 0.9718831612650204
MLP vs SVM on digits ->
  MLP -> Acc 0.9722, F1 0.9719, ROC-AUC 0.9983
  SVM -> Acc 0.9750, F1 0.9749, ROC-AUC 0.9995
PCA-based shallow NN acc: 0.9638888888888889  raw-pixel NN acc: 0.9722222222222222

Saved plots (lab10_plots/):
lab10_plots/loss.png
lab10_plots/valacc.png
lab10_plots/sample_preds.png
lab10_plots/sin_fit.png
lab10_plots/loss.png
lab10_plots/sequence_similarity.png
lab10_plots/roc_class0.png
lab10_plots/feature_comp.png
